In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import make_scorer
import graphviz



from xgboost import XGBClassifier
import xgboost as xgb
from imblearn.over_sampling import SMOTE

import seaborn as sns

In [3]:
#Ideas:
#Use a custom objective function (XGBoost) to maximize profits directly 
#i.e. not caring about the accuracy of the predictions

#Change odds from H D A to sum H/D H/A
#Try it you will like it.

In [4]:
#Project steps:
#1
#Extract relevant features from football-data.co.uk. Do some feature engineering.
#Expect some improvement from competence_difference. Better formula may help
#Do we need more data?

#2
#Predict outcomes of each match. Training using data since 2009. Leave 2015 and 2016.
#2015 is for cross-validation of classifiers. 2016 is for testing 

#3
#Estimate probability that prediction is correct. In other words, how confident are we?

#4 
#Using all the information gathered, formulate a betting strategy

In [135]:
f9 = r'/home/william/Dropbox/Kaggle/Soccer/season9.csv'
f10 = r'/home/william/Dropbox/Kaggle/Soccer/season10.csv'
f11 = r'/home/william/Dropbox/Kaggle/Soccer/season11.csv'
f12 = r'/home/william/Dropbox/Kaggle/Soccer/season12.csv'
f13 = r'/home/william/Dropbox/Kaggle/Soccer/season13.csv'
f14 = r'/home/william/Dropbox/Kaggle/Soccer/season14.csv'
f15 = r'/home/william/Dropbox/Kaggle/Soccer/season15.csv'
f16 = r'/home/william/Dropbox/Kaggle/Soccer/season16.csv'


df9 = pd.read_csv(f9)
df10 = pd.read_csv(f10)
df11 = pd.read_csv(f11)
df12 = pd.read_csv(f12)
df13 = pd.read_csv(f13)
df14 = pd.read_csv(f14)
df15 = pd.read_csv(f15)
df16 = pd.read_csv(f16)

#df16['sum_odds'] = df16['B365H']+df16['B365A']+df16['B365D']
#df16['H'] = df16['B365H']/df16['B365D']
#df16['A'] = df16['B365A']/df16['B365D']
#df16 = df16.drop(['B365H', 'B365D', 'B365A'],axis=1)


df_test = df16

df_train = df9.append([df10,df11,df12,df13,df14,df15],ignore_index=True)
#df_train['sum_odds'] = df_train['B365H']+df_train['B365A']+df_train['B365D']
#df_train['H'] = df_train['B365H']/df_train['B365D']
#df_train['A'] = df_train['B365A']/df_train['B365D']
#df_train = df_train.drop(['B365H', 'B365D', 'B365A'],axis=1)

f_training = r'/home/william/Dropbox/Kaggle/Soccer/training.csv'
f_testing = r'/home/william/Dropbox/Kaggle/Soccer/testing.csv'

#df_train = pd.read_csv(f_training)
#df_test = pd.read_csv(f_testing)

#df_train = df_train.drop('B365A',axis=1)
#df_test = df_test.drop('B365A',axis=1)


train_X = df_train.drop('Home/Away win',axis=1)
train_y = df_train['Home/Away win']+1





test_X = df_test.drop('Home/Away win',axis=1)
test_y = df_test['Home/Away win']+1





In [136]:
clf = XGBClassifier()
clf.fit(train_X,train_y)
clf.score(test_X,test_y)

0.60263157894736841

In [137]:
dtrain = xgb.DMatrix(train_X,label=train_y)
dtest = xgb.DMatrix(test_X,label=test_y)


In [146]:
params = {''}
clf = xgb.train()

In [147]:
np.average((first.predict(dtest)==test_y))

0.60789473684210527

In [ ]:

print ('start running example to used customized objective function')

dtrain = xgb.DMatrix('../data/agaricus.txt.train')
dtest = xgb.DMatrix('../data/agaricus.txt.test')

# note: for customized objective function, we leave objective as default
# note: what we are getting is margin value in prediction
# you must know what you are doing
param = {'max_depth': 2, 'eta': 1, 'silent': 1}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 2

# user define objective function, given prediction, return gradient and second order gradient
# this is log likelihood loss
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0-preds)
    return grad, hess

# user defined evaluation function, return a pair metric_name, result
# NOTE: when you do customized loss function, the default prediction value is margin
# this may make builtin evaluation metric not function properly
# for example, we are doing logistic loss, the prediction is score before logistic transformation
# the builtin evaluation error assumes input is after logistic transformation
# Take this in mind when you use the customization, and maybe you need write customized evaluation function
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

# training with customized objective, we can also do step by step training
# simply look at xgboost.py's implementation of train
bst = xgb.train(param, dtrain, num_round, watchlist, logregobj, evalerror)